# Biber Resultate Pre-Processing

In [ ]:
import pandas as pd

## Excel to csv und df to Excel/csv

In [ ]:
excel_file = "data/Biber.xlsx"
data_2024 = pd.read_excel(excel_file, sheet_name="2024")
data_2025a = pd.read_excel(excel_file, sheet_name="2025_09_10")
data_2025b = pd.read_excel(excel_file, sheet_name="2025_11_12_13")

In [ ]:
data_2024.to_csv("data/2024.csv", index=False)
data_2025a.to_csv("data/2025a.csv", index=False)
data_2025b.to_csv("data/2025b.csv", index=False)

In [ ]:
#data_2024 = pd.read_csv("data/2024.csv")
#data_2025a = pd.read_csv("data/2025a.csv")
#data_2025b = pd.read_csv("data/2025b.csv")

## Daten zusammenführen

In den verschiedenen Jahren gibt es unterschiedliche Aufgaben. Wir überführen die Daten deshalb in ein anderes Format - das sogenannte Long-Format - so dass alle Jahre in den selben Dataframe gespeichert und dann gemeinsam analysiert werden können.

|Jahr|Klassenstufe|Klasse|Aufgabe|Punkte|
|--|--|--|--|--|
|2024|Schuljahre 11, 12 und 13|KHR|Happy Birthday|-2|
|...|...|...|...|...|

In [ ]:
data_2024_long = pd.melt(
    data_2024,
    id_vars=['Jahr', 'Klassenstufe', 'Klasse'],     # columns that stay as identifiers
    var_name='Aufgabe',   # name for the new "variable" column
    value_name='Punkte'     # name for the new "value" column
)

In [ ]:
data_2025a_long = pd.melt(
    data_2025a,
    id_vars=['Jahr', 'Klassenstufe', 'Klasse'],
    var_name='Aufgabe',
    value_name='Punkte'
)

In [ ]:
data_2025b_long = pd.melt(
    data_2025b,
    id_vars=['Jahr', 'Klassenstufe', 'Klasse'],
    var_name='Aufgabe',
    value_name='Punkte'
)

Jetzt müssen wir die drei Dataframes für die Analyse zusammenführen. In diesem Fall also einfach untereinander schreiben, da sie die gleichen Spaltennamen haben.

In [ ]:
df = pd.concat([data_2024_long, data_2025a_long, data_2025b_long])

Wir entfernen die Bonuspunkte. Diese Punkte bekommt jede(r) Teilnehmer(in) und sie dienen nur dazu, dass kein Teilnehmer weniger als 0 Punkte erreichen kann. Wir benötigen diese nicht für die Analyse - im Gegenteil, die Bonuspunkte stören sogar.

In [ ]:
df = df[df["Aufgabe"] != "Bonus"]

In [ ]:
df.describe()

In [ ]:
df.to_csv("data/Biber-Daten-Long.csv", index=False)
df.to_excel("data/Biber-Daten-Long.xlsx", index=False)

und nochmal im Wide-Format. Dazu müssen wir einen eindeutigen Zeilenindex hinzufügen, ansonsten müssten wir `pivot_table` verwenden und Daten aggregieren - was wir ja nicht wollen beim Rohdatenexport.

In [ ]:
df.loc[:, "row_id"] = df.groupby(["Jahr", "Klassenstufe", "Klasse"]).cumcount()

In [ ]:
df_wide = df.pivot(
    index=["Jahr", "Klassenstufe", "Klasse", "row_id"], # neue Zeilen
    columns="Aufgabe",
    values="Punkte",
).reset_index()

df_wide.drop(["row_id"], axis=1, inplace=True)

In [ ]:
df_wide.describe()

In [ ]:
df_wide

In [ ]:
df_wide.to_csv("data/Biber-Daten-Wide.csv", index=False)
df_wide.to_excel("data/Biber-Daten-Wide.xlsx", index=False)

### Bilder als Dateien abspeichern

In [ ]:
img = df.boxplot(column='Punkte', by='Jahr', showmeans=True, grid=False)
img.get_figure().savefig('img/a_boxplot.svg')
img.get_figure().savefig('img/a_boxplot.png')